# Deduper Script Testing

### Inline Testing

In [63]:
import re

#####################################################################################
##### Define higher order functions #################################################
#####################################################################################

def strand_checker(flag):
    '''Takes the bitwise flag and checks it for strandedness. Assumes read is mapped, otherwise returnes None.
    Assumes data are single-end. Returns "+" or "-", depending on strand.'''
    # Check read is mapped
    if ((flag & 4) == 4):
        return None
    # Assume positive strand
    strand = "+"
    if ((flag & 16) == 16):
        # Changes strand if bit 16 is set
        strand = "-"
    return strand

def POS_correct(cigar_string, POS):
    '''Takes the CIGAR string and corrects the start position in the SAM file (POS)
    according to any soft clipping at the beginning of the CIGAR string. Returns the corrected POS value.'''
    search = re.search(r"^\d+S", cigar_string)
    if search:
        soft_clip = int(search.group(0)[:-1])
        corrected_POS = POS - soft_clip
    else:
        corrected_POS = POS
        # Corrected starting position could be the same as the starting position
    return corrected_POS

def count_lines(infile):
    '''Opens the input file and returns the number of lines in the file.'''
    with open(infile) as file:
        for i, line in enumerate(file):
            pass
    return i + 1

#####################################################################################
##### Set Up ########################################################################
#####################################################################################

### Initialize empty dictionaries for tuples and dictionaries
tuple_dict = {}
umi_dict = {}

### Make UMI dictionary
with open("umi_list.txt") as umi_list:
    for umi in umi_list:
        umi = umi.strip("\n")
        umi_dict[umi] = 0
print("Created UMI dictionary.", flush = True)

###Initialze chromosome variable for chromosome check in the with loop
chromosome = 1

### Create output filename
infile = "/Users/sally_claridge/Desktop/deduper-sclaridg/test_input.sam"
filename = infile.split("/")[-1].split(".")[0]   # Isolate file name
deduped = "".join([filename, "_deduped"])      # Add "deduped"
outfile = infile.replace(filename, deduped)       # Replace in original filepath

### Get file linecount
total_lines = count_lines(infile)

print("Opening SAM files for reading and writing.", flush = True)

#####################################################################################
##### Read through SAM file #########################################################
#####################################################################################

with open(infile, "r+") as file, open(outfile, "w+") as out:
    linecount = 0
    for line in file:
        linecount += 1
        if line.startswith("@") == True:
            pass
        elif line.startswith("@") == False:
            line_list = line.strip("\n").split("\t")
            
            ### Progress report
            if linecount % 200000 == 0:
                print("Passed line" + str(linecount) + ".", flush = True)
            
            ### Check UMI
            umi = line_list[0].split(":")[-1]
            if umi in umi_dict:
                umi_dict[umi] +=1
            else:
                if linecount == total_lines:
                    for value in tuple_dict.values():
                        out.write(value)
                continue
                
            ### Check bitwise flag
            flag = int(line_list[1])
            strand = strand_checker(flag)
            
            ### Check chromosome
            prev_chromosome = chromosome
            chromosome = line_list[2]
            if chromosome != prev_chromosome:
                for value in tuple_dict.values():
                    out.write(value)
                tuple_dict = {}
            
            ### Check POS and CIGAR string
            POS = int(line_list[3])
            cigar_string = line_list[5]
            new_POS = POS_correct(cigar_string, POS)
            
            ### Calculate average quality score across the alignment
            quality = line_list[10]
            score_total = 0
            for char in quality:
                score_total += ord(char) - 33 # Convert ASCII score to phred score
            score = score_total / len(quality)    
            
            ### Check tuple_dict
            query_tuple = (umi, strand, chromosome, new_POS)
            if query_tuple in tuple_dict:
                existing_quality = tuple_dict[query_tuple].split("\t")[10]
                existing_score_total = 0
                for char in existing_quality:
                    existing_score_total += ord(char) - 33 # Convert ASCII score to phred score
                existing_score = existing_score_total / len(existing_quality)
                if existing_score >= score:
                    pass
                elif existing_score < score:
                    tuple_dict[query_tuple] = line
            if query_tuple not in tuple_dict:
                tuple_dict[query_tuple] = line
                
            ### Last line check
            if linecount == total_lines:
                for value in tuple_dict.values():
                    out.write(value)

print("Finished.", flush = True)
#exit

Created UMI dictionary.
Opening SAM files for reading and writing.
Finished.


### Syntax with `argparse`

In [ ]:
#!/usr/bin/env python
#claridge_deduper.py

import argparse
import re

parser = argparse.ArgumentParser(description="Removes PCR duplicates from a SAM file of uniquely mapped, single-end reads. Duplicates with the highest per-base average quality are printed to an output file.")
parser.add_argument('-f','--file', help='absolute path to SAM file to be deduped.', required=True, type=str)
parser.add_argument('-p','--paired', help='reads are paired-end.', required=False, action='store_true', type=str)
parser.add_argument('-u','--umi', help='absolute path to list of UMIs (do not set flag if randomers were used).', required=False, type=str)
args = parser.parse_args()

if args.paired:
    print("No paired-end functionality at this time.")
    exit

if args.umi == False:
    print("Need UMI list to continue.")
    exit
elif args.umi:
    with open(args.umi) as umi_list:
        
        
with open(args.file) as file: